<a href="https://colab.research.google.com/github/nivalf/Predict_Energy_Usage/blob/main/predict_energy_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Predict Energy Usage

## Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed, as well as Scikit-Learn ≥0.20.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "predict_diamond_price"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Load the Data

In [ ]:
import pandas as pd

def load_energy_usage_data():
    return pd.read_csv('https://raw.githubusercontent.com/nivalf/Predict_Energy_Usage/main/data/dataset.csv')

In [ ]:
energy_usage = load_energy_usage_data()
energy_usage.head()

<ipython-input-6-71a860936688>:1: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  energy_usage = load_energy_usage_data()


,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,Weather icon,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,clear-night,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
1,1451624410,1.585083,0.003417,1.585083,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,...,clear-night,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,1451624420,0.533817,0.003450,0.533817,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,...,clear-night,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,1451624430,1.190950,0.003083,1.190950,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,...,clear-night,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1451624440,1.582083,0.003067,1.582083,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,...,clear-night,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


hello world